# Generate Power Functional Connectivity
This script performs further processing on the data by generating the functional connectivities from the preprocessed data. We take in data from the final output of AA in the `aamod_firstlevel_modelestimate_saveresids_00001` folder and generate functional connectivity from the NIfTI files using the Power et al. brain atlas composed of 264 ROIs.

I'm following the guide here: <https://nilearn.github.io/auto_examples/03_connectivity/plot_sphere_based_connectome.html>

## Set up imports

In [1]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import time

from nilearn import plotting, image, datasets
from nilearn.input_data import NiftiSpheresMasker
from nilearn.connectome import ConnectivityMeasure
import numpy as np

## Set up input and output

In [2]:
aa_folder = 'BioBank_Analysis/'
saveresids_folder = 'aamod_firstlevel_modelestimate_saveresids_00001/'
python_fc_folder = 'python_power_fc'

## Set up FC functions

In [3]:
power = datasets.fetch_coords_power_2011()
coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T

spheres_masker = NiftiSpheresMasker(
    seeds=coords, smoothing_fwhm=6, radius=5., standardize=True)

def generate_power_fc(file):
    time_series = spheres_masker.fit_transform(file)
    
    correlation_measure = ConnectivityMeasure(kind='correlation')
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    
    return correlation_matrix

In [4]:
def visualize_corr_matrix(corr_matrix):
    np.fill_diagonal(corr_matrix, 0)
    plotting.plot_matrix(corr_matrix, vmin=-1., vmax=1., colorbar=True, figure=(20, 20),
                     title='Power correlation matrix')
#     plotting.plot_connectome(corr_matrix, coords, title='Power correlation graph',
#                          edge_threshold='98.0%', node_size=20, colorbar=True)

## Generate FCs

In [7]:
for age in range(18, 19):
    print("Processing age:", age)
    age_dir = f'/imaging2/owenlab/wilson/MovieData/Age{age}'

    input_path = join(age_dir, aa_folder, saveresids_folder)
    output_path = join(age_dir, aa_folder, python_fc_folder)

    # Grab all subjects
    subjects = [folder for folder in listdir(input_path) if folder.startswith("sub-")]

    for subject in subjects:
        print("  Processing subject:", subject)
        start_time = time.time()
        subject_input_path = join(input_path, subject)
        subject_output_path = join(output_path, subject)

        # Check if we've already generated the FC for this subject
        if exists(subject_output_path):
            print("  Skipping subject due to already generated FC.")
            continue

        subject_niftis = [join(subject_input_path, f) for f in listdir(subject_input_path) 
                          if isfile(join(subject_input_path, f)) and f.startswith('ResI')]

        if len(subject_niftis) != 745:
            print("  Incomplete subject niftis:", len(subject_niftis))
            continue

        subject_concat_image = image.concat_imgs(subject_niftis)
        subject_corr_matrix = generate_power_fc(subject_concat_image)

        makedirs(subject_output_path)
        subject_corr_filename = join(subject_output_path, 'power_fc.npy')
        np.save(subject_corr_filename, subject_corr_matrix)
        print("  Time elapsed:", time.time() - start_time)
    #     visualize_corr_matrix(subject_corr_matrix)

Processing age: 18
  Processing subject: sub-NDARRJ989YPA
  Time elapsed: 95.42229318618774
  Processing subject: sub-NDAREP505XAD
  Time elapsed: 94.71648669242859
  Processing subject: sub-NDARFV061AR5
  Time elapsed: 94.10099744796753
  Processing subject: sub-NDARMR039JV5
  Time elapsed: 94.61761903762817
  Processing subject: sub-NDARGB441VVD
  Time elapsed: 93.4177131652832
  Processing subject: sub-NDARPN886HH9
  Time elapsed: 93.43469619750977
  Processing subject: sub-NDAREL410HHK
  Time elapsed: 93.80072379112244
  Processing subject: sub-NDARYF728BEP
  Time elapsed: 97.86800289154053
  Processing subject: sub-NDARKM784VA7
  Time elapsed: 120.15167307853699
  Processing subject: sub-NDARPY478YM0
  Time elapsed: 93.65082240104675
  Processing subject: sub-NDARYF368WEA
  Time elapsed: 97.21884632110596
  Processing subject: sub-NDARUV578DU2
  Incomplete subject niftis: 0
  Processing subject: sub-NDARHL164JHH
  Time elapsed: 94.91461110115051
  Processing subject: sub-NDARZP630